In [4]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
# Load dataset 
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
sample = pd.read_csv('input/sampleSubmission.csv')

# weather = pd.read_csv('input/weather.csv')
# spray = pd.read_csv('input/spray.csv')

In [37]:
train.dtypes

Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
dtype: object

In [38]:
test.dtypes

Id                          int64
Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
dtype: object

In [52]:
def generate_date_features(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].apply(lambda x: x.year)
    df['Month'] = df['Date'].apply(lambda x: x.month)
    df['DayOfMonth'] = df['Date'].apply(lambda x: x.day)
    # df['DayOfWeekName'] = df['Date'].apply(lambda x: x.day_name())
    df['DayOfWeek'] = df['Date'].apply(lambda x: x.dayofweek)
    df['DayOfYear'] = df['Date'].apply(lambda x: x.dayofyear)
    df['WeekOfYear'] = df['Date'].apply(lambda x: x.weekofyear)
    df['IsLeapYear'] = df['Date'].apply(lambda x: x.is_leap_year)
    df['IsLeapYear'] = df['IsLeapYear'].astype(str)
    df['Quarter'] = df['Date'].apply(lambda x: x.quarter)

def transform_df(df):
    df_ = df.copy()
    df_['Lat_int'] = df_.Latitude.astype(int)
    df_['Long_int'] = df_.Longitude.astype(int)
    generate_date_features(df_)
    return df_

In [67]:
train2 = transform_df(train)
test2 = transform_df(test)

In [68]:
# drop address columns
train2 = train2.drop(['Date','Address', 'AddressNumberAndStreet','WnvPresent', 'NumMosquitos'],  axis = 1)
test2 = test2.drop(['Date','Id', 'Address', 'AddressNumberAndStreet'],axis = 1)

In [69]:
train2.dtypes 

Species              int64
Block                int64
Street               int64
Trap                 int64
Latitude           float64
Longitude          float64
AddressAccuracy      int64
Lat_int              int64
Long_int             int64
Year                 int64
Month                int64
DayOfMonth           int64
DayOfWeek            int64
DayOfYear            int64
WeekOfYear           int64
IsLeapYear          object
Quarter              int64
dtype: object

# label encoder

In [70]:
from sklearn import ensemble, preprocessing

In [71]:
# Convert categorical data to numbers
lbl = preprocessing.LabelEncoder()
lbl.fit(list(train2['Species'].values) + list(test2['Species'].values))
train2['Species'] = lbl.transform(train2['Species'].values)
test2['Species'] = lbl.transform(test2['Species'].values)

lbl.fit(list(train2['Street'].values) + list(test2['Street'].values))
train2['Street'] = lbl.transform(train2['Street'].values)
test2['Street'] = lbl.transform(test2['Street'].values)

lbl.fit(list(train['Trap'].values) + list(test['Trap'].values))
train2['Trap'] = lbl.transform(train2['Trap'].values)
test2['Trap'] = lbl.transform(test2['Trap'].values)

lbl.fit(list(train2['IsLeapYear'].values) + list(test2['IsLeapYear'].values))
train2['IsLeapYear'] = lbl.transform(train2['IsLeapYear'].values)
test2['IsLeapYear'] = lbl.transform(test2['IsLeapYear'].values)

In [73]:
# Get labels
labels = train.WnvPresent.values

In [85]:
# Random Forest Classifier 
clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=100, min_samples_split=1.0)
clf.fit(train2, labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=1.0,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [86]:
# create predictions and submission file
# not a good training by itself
predictions = clf.predict(test2)
sample['WnvPresent'] = predictions 
# wait why is it not binary prediction, all zeros!
sample.WnvPresent.value_counts()

0    116293
Name: WnvPresent, dtype: int64

In [78]:
sample.to_csv('preds_v1_train_only.csv', index=False)